<a href="https://colab.research.google.com/github/erico-bachour/Challenge_Data_Science_2_Alura/blob/main/Challenge_Data_Science_2_InsightPlaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Estudo para a Imobiliária **Insight Places**

Com a base de dados fornecida pela Inisight Places, temos o objetivo de criar um modelo de regressão para precificar imóveis e um recomendador de imóveis. Esses por sua vez, auxiliarão a empresa a otimizar seus valores de venda e aluguel de imóveis, dessa forma, aumentando seu potencial de receita e subsequentemente de lucro.

In [1]:
#importando bibliotecas básicas para análise de dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sc
import statsmodels.api as sm
%matplotlib inline
sns.set_style('darkgrid')

In [2]:
#importando a biblioteca PySpark
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

Iniciando uma SparkSession

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [6]:
spark

Acessando a SparkUI

In [7]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip -y

Archive:  ngrok-stable-linux-amd64.zip
caution: filename not matched:  -y


In [8]:
get_ipython().system_raw('./ngrok authtoken 2HrdUIS1XiYY5fV2twMmahQ1UML_3PMhmQFai65KVyZRcJ3ZC')
get_ipython().system_raw('./ngrok http 4050 &')

In [9]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


Baixando o arquivo .zip para acesso à base de dados

In [10]:
print('Baixando base de dados...Baixado!')
!wget https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip

Baixando base de dados...Baixado!
--2022-11-30 18:32:26--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 54.231.194.73, 52.217.232.241, 52.216.140.156, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|54.231.194.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip.4’

semana-1.zip.4      100%[===================>]  18.10M  58.2MB/s    in 0.3s    

2022-11-30 18:32:27 (58.2 MB/s) - ‘semana-1.zip.4’ saved [18975214/18975214]



In [11]:
!unzip -q semana-1.zip -y

caution: filename not matched:  -y


In [12]:
path = '/content/dataset_bruto.json'
dados = spark.read.json(path)

Contando quantos registros temos na base de dados:

In [13]:
dados.count()

89083

Visualisando como está montada a base de dados:

In [14]:
dados.limit(5).toPandas()

,anuncio,imagens,usuario
0,"(0, [], [16], [0], [], (Centro, 20061003, Rio ...","[(39d6282a-71f3-47bc-94aa-909351ecd881, https:...","(9d44563d-3405-4e84-9381-35b7cf40a9a4, Frank)"
1,"(0, [], [14], [0], [], (Centro, 20051040, Rio ...","[(23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https:...","(36245be7-70fe-40cd-84d4-70a57dc8f976, Caroline)"
2,"(0, [1026], [1026], [0], [], (Maria da Graça, ...","[(1da65baa-368b-490f-bef7-1834b9a428c1, https:...","(9dc415d8-1397-4d8d-8425-b8310fad309d, Oliver)"
3,"(0, [120], [120], [0], [Portão eletrônico, Con...","[(79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https:...","(9911a2df-f299-4a73-a384-281417abaf69, Matthew)"
4,"(0, [3], [3], [0], [], (São Cristóvão, 2093167...","[(e2bc497b-6510-4731-8942-97b3d3cdab5e, https:...","(240a7aab-12e5-4069-9a2c-9dca4c4f9c68, Jude)"


In [15]:
dados.columns

['anuncio', 'imagens', 'usuario']

In [16]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

Como queremos lidar somente com as variáveis da coluna "anuncio", vamos selecioná-la.

| Colunas         | Descrição                                                      |
|-----------------|----------------------------------------------------------------|
| id              | Código de identificação do anúncio no sistema da InsightPlaces |
| tipo_unidade    | Tipo de imóvel (apartamento, casa e outros)                    |
| tipo_uso        | Tipo de uso do imóvel (residencial ou comercial)               |
| area_total      | Área total do imóvel (construção e terreno)                    |
| area_util       | Área construída do imóvel                                      |
| quartos         | Quantidade de quartos do imóvel                                |
| suites          | Quantidade de suítes do imóvel                                 |
| banheiros       | Quantidade de banheiros do imóvel                              |
| vaga            | Quantidade de vagas de garagem do imóvel                       |
| caracteristicas | Listagem de características do imóvel                          |
| andar           | Número do andar do imóvel                                      |
| endereco        | Informações sobre o endereço do imóvel                         |
| valores         | Informações sobre valores de venda e locação dos imóveis       |

In [17]:
dados\
    .select(
        'anuncio.*'
    )\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

In [18]:
anuncio = dados.select('anuncio.*')

Assim como requisitado pelo time de Data Science da imobiliária, iremos filtrar os imóveis por tipo de uso, tipo de unidade e tipo de anuncio. Mais especificamente os imóveis nas categorias **residencial, apartamento e usado**.


In [19]:
anuncio\
      .select('tipo_uso')\
      .groupBy('tipo_uso')\
      .count()\
      .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



Como imóveis residenciais e comerciais seguem métricas diferentes no que se refere a precificação, ficaremos somente com os residenciais em nosso modelo, por enquanto.

In [20]:
anuncio = anuncio.filter('tipo_uso == "Residencial"')

In [21]:
anuncio\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|84541|
+-----------+-----+



Passando para o tipo de unidade, vamos agrupar para ver como estão distribuídos os imóveis em nossa base de dados

In [22]:
anuncio\
      .groupBy('tipo_unidade')\
      .count()\
      .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



Agora vamos filtrar novamente para ficarmos somente com os **Apartamentos**

In [23]:
anuncio = anuncio.filter('tipo_unidade == "Apartamento"')

In [24]:
anuncio\
      .groupBy('tipo_unidade')\
      .count()\
      .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66797|
+------------+-----+



Antes da úlitma filtragem, vamos ver como eles estão distribuídos no que tange a idade.

In [25]:
anuncio\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



Filtrando mais uma vez, vamos ficar somente com os imóveis **usados**.

In [26]:
anuncio = anuncio.filter('tipo_anuncio == "Usado"')

In [27]:
anuncio\
      .groupBy('tipo_anuncio')\
      .count()\
      .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



Com a base de dados filtrada como a equipe de Data Science solicitou, partiremos para mais uma etapa do ajuste de nossa base de dados.

In [28]:
anuncio.show(truncate=False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

In [29]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

Como pudemos ver no printSchema acima, algumas colunas estão em formato de listas quando deveriam estar em formato numérico. Dessa forma vamos partir para essas conversões.

In [30]:
from pyspark.sql import functions as f

Antes de fazermos as conversões, vamos nos assegurar que as listas só possuem um elemento.

In [31]:
anuncio\
    .select(f.size(f.col('quartos')).alias('quartos'))\
    .groupBy('quartos')\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+



In [32]:
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [33]:
for col in caracteristicas:
  anuncio.select(f.size(f.col(col)).alias(col))\
  .groupBy(col)\
  .count()\
  .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



Como pudemos ver acima, existem elementos vazios em três colunas: vaga, suites e area total. Nas duas primeiras é compreensível, uma vez que existem imóveis que não as possuem. Já no caso de área total, é preciso uma investigação.

Passada a etapa de averiguação, vamos partir para a substituição das listas pelos valores nas colunas.

In [34]:
anuncio\
      .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])\
      .show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [35]:
anuncio=anuncio.select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])

In [36]:
anuncio.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [37]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

Como a imobiliária Insight Places atua somente no Rio de Janeiro, a equipe de Data Science nos solicitou uma nova filtragem, agora por cidade, bairro e zona.

In [38]:
anuncio\
    .select('endereco.*')\
    .show()

+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|
+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43.634034|  BR|Estrada de Santa ...|Zona Oeste|
|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.928108|-43.635375|  BR|Estrada de Santa ...|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948756|-43.582824|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|Rio de Janeiro|Rio de Janeiro|-22.888194|-43.629602|  BR|Estrada da Paciência|Zona Oeste|
|           Guaratiba|23036060|Rio de Janeiro|Rio de Janeiro|-22.948291|-43.582205|  BR|Estrada Cabuçu de...|Zona Oeste|
|              Cosmos|23066271|R

In [39]:
anuncio\
    .select('endereco.bairro', 'endereco.zona')\
    .show()

+--------------------+----------+
|              bairro|      zona|
+--------------------+----------+
|           Paciência|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|  Pedra de Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|        Campo Grande|Zona Oeste|
|          Santa Cruz|Zona Oeste|
|           Guaratiba|Zona Oeste|
|Recreio dos Bande...|Zona Oeste|
|              Cosmos|Zona Oeste|
|            Realengo|Zona Oeste|
|              Cosmos|Zona Oeste|
|     Todos os Santos|Zona Norte|
+--------------------+----------+
only showing top 20 rows



In [40]:
anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|[P

In [41]:
anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')\
    .show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|[Academia, Churra...|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|[Churrasqueira, P...|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|[P

Excluindo a coluna de endereço para ficarmos somente com as informações de bairro e zona

In [42]:
anuncio=anuncio\
    .select('*',  'endereco.bairro', 'endereco.zona')\
    .drop('endereco')

In [43]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



Agora vamos focar na coluna 'valores', mais especificamente, em transformar o conteúdo dela, que está em formato de array, em colunas separadas em nosso DataFrame.

Analisando o .printSchema acima, podemos ver que a coluna 'valores' esta organizada em um array, ou seja, uma lista que contém elementos. Esses elementos são dicionários com as informações de **condomínio, iptu, tipo e valor**.

Para isso vamos utilizar a funcção .explode() do spark.

Como no momento do cadastro, o vendedor tem a opção de anuniciar para venda e/ou aluguel, alguns anuncios possuem esse valor a mais, por causa do aluguel.

Para o modelo, usaremos somente os valores de **venda**, pois estamos desenvolvendo um modelo de precificação dos imóveis.

In [44]:
anuncio\
    .select('valores')\
    .show()

+--------------------+
|             valores|
+--------------------+
|[{245, null, Vend...|
|[{0, 0, Venda, 15...|
|[{0, 0, Venda, 20...|
|[{285, null, Vend...|
|[{245, null, Vend...|
|[{285, null, Vend...|
|[{250, null, Vend...|
|[{245, null, Vend...|
|[{245, null, Vend...|
|[{240, null, Vend...|
|[{0, 0, Venda, 15...|
|[{240, 0, Venda, ...|
|[{245, null, Vend...|
|[{290, null, Vend...|
|[{285, null, Vend...|
|[{0, 0, Venda, 30...|
|[{null, null, Ven...|
|[{280, 0, Venda, ...|
|[{280, null, Vend...|
|[{0, 0, Venda, 26...|
+--------------------+
only showing top 20 rows



In [45]:
anuncio\
      .select(f.explode('valores').alias('valores'))\
      .select('valores.*')\
      .filter('valores.tipo=="Venda"')\
      .count()

66562

In [46]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .show(50, truncate=False)

+------------------------------------+----------+----+-------+-----+
|id                                  |condominio|iptu|tipo   |valor|
+------------------------------------+----------+----+-------+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda  |15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda  |15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda  |20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda  |20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda  |15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda  |20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda  |15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda  |15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda  |15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda  |17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda  |15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12

In [47]:
anuncio\
    .select('id', f.explode('valores').alias('valores'))\
    .select('id', 'valores.*')\
    .filter('valores.tipo=="Venda"')\
    .show(truncate=False)

+------------------------------------+----------+----+-----+-----+
|id                                  |condominio|iptu|tipo |valor|
+------------------------------------+----------+----+-----+-----+
|d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3|245       |null|Venda|15000|
|085bab2c-87ad-452a-bd0f-8b1451509f84|0         |0   |Venda|15000|
|18d22cbe-1b86-4764-8def-70c615f15a61|0         |0   |Venda|20000|
|bed8a354-9317-4426-b27e-1c798f864271|285       |null|Venda|20000|
|12a13315-d67f-48f0-9497-017b83252781|245       |null|Venda|15000|
|a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e|285       |null|Venda|20000|
|a6e3173b-c950-4db4-9b7b-80c44bd75e90|250       |null|Venda|15000|
|2e6e5dfb-206c-4968-944b-ea4c3918b50d|245       |null|Venda|15000|
|99f8d0f9-95a4-4613-a55d-c949e7a73e90|245       |null|Venda|15000|
|b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8|240       |null|Venda|17999|
|dc99d9e1-4c63-41f7-a53d-678b2b3ccb52|0         |0   |Venda|15000|
|aa3606d8-8bcd-45ec-a609-d68a58a12014|240       |0   |Venda|19

In [48]:
valores = anuncio\
      .select('id', f.explode('valores').alias('valores'))\
      .select('id', 'valores.*')\
      .filter('valores.tipo=="Venda"')

Utilizando a coluna 'id' como chave, vamos unir o dataframe 'valores' com o restante do DataFrame 'anuncio'.

In [49]:
anuncio.join(valores, 'id', how='inner').count()

66562

In [50]:
anuncio.join(valores, 'id', how='inner').show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+----------+-----+-----+--------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|condominio| iptu| tipo|   valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+----------+-----+-----+--------+
|006e2b19-f4de-450...|    0|       116|      116|        2|[Elevador, Academ...|      3|     1|       Usado| Apartamento|Residencial|   1|[{100, 100, Venda...|     Barra da Tijuca|Zona Oeste|       100|  100|Venda| 3793260|
|011acce0-129f-4c2...|    0|       143|      143|        4|[Churrasqueira, P...|      4|     3|       Us

Excluindo agora a coluna 'valores'

In [51]:
anuncio=anuncio.join(valores, 'id', how='inner').drop('valores')

anuncio.show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+-----+-----+--------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|      zona|condominio| iptu| tipo|   valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+-----+-----+--------+
|006e2b19-f4de-450...|    0|       116|      116|        2|[Elevador, Academ...|      3|     1|       Usado| Apartamento|Residencial|   1|     Barra da Tijuca|Zona Oeste|       100|  100|Venda| 3793260|
|011acce0-129f-4c2...|    0|       143|      143|        4|[Churrasqueira, P...|      4|     3|       Usado| Apartamento|Residencial|   1|         Jacarepaguá|Zona Oeste|      3948| 2000|V

Com a base de dados tratada, vamos salvá-la em formato parquet e .csv.

In [52]:
anuncio.write.parquet(
    path = '/content/drive/Challenge_Data_Science_2/parquet',
    mode = 'overwrite'
)

Para podermos salvar em .csv devemos excluir  a coluna 'caracteristicas', pois o formato não aceita array como datatype.







In [53]:
anuncio = anuncio.drop("caracteristicas")

In [54]:
   anuncio.write.csv(
    path = '/content/drive/Challenge_Data_Science_2/csv',
    mode = 'overwrite',
    header=True
   )

Comparativo de tempo de execução dos arquivos .parquet e .csv

In [58]:
#lendo arquivos

anuncio_parquet = spark.read.parquet('/content/drive/Challenge_Data_Science_2/parquet')

anuncio_csv = spark.read.csv('/content/drive/Challenge_Data_Science_2/csv', header = True)

In [59]:
%%time
anuncio_parquet.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|              Gávea|  597|
|       Tomás Coelho|   38|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|              Rocha|   38|
|      Bento Ribeiro|   33|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|        Jacarepaguá| 4437|
|           Botafogo| 3499|
|               Leme|  234|
|       Campo Grande|  639|
|       Padre Miguel|   15|
|           Flamengo| 1378|
|       Santo Cristo|  357|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 17.5 ms, sys: 3.38 ms, total: 20.9 ms
Wall time: 2.18 s


In [60]:
%%time
anuncio_csv.groupBy('bairro').count().show()

+-------------------+-----+
|             bairro|count|
+-------------------+-----+
|             Cocotá|   16|
|              Gávea|  597|
|       Tomás Coelho|   38|
|            Ipanema| 2113|
|           Realengo|   46|
|      Gardênia Azul|    7|
|              Rocha|   38|
|      Bento Ribeiro|   33|
|Vicente de Carvalho|   86|
|         Manguinhos|    1|
|        Jacarepaguá| 4437|
|           Botafogo| 3499|
|               Leme|  234|
|       Campo Grande|  639|
|       Padre Miguel|   15|
|           Flamengo| 1378|
|       Santo Cristo|  357|
|  Engenho da Rainha|   25|
| Pedra de Guaratiba|   12|
|            Piedade|  104|
+-------------------+-----+
only showing top 20 rows

CPU times: user 13.5 ms, sys: 1.21 ms, total: 14.7 ms
Wall time: 1.67 s
